In [3]:
import numpy as np 
import math
import pandas as pd 
import pandas_datareader as pdr 
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.models import sequential
from keras.layers import Dense , LSTM
from keras.models import Sequential
plt.style.use("fivethirtyeight")
import datetime
from datetime import  timedelta



#'NTPC.NS','HEROMOTOCO.NS','TECHM.NS'	,'COALINDIA.NS','APOLLOHOSP.NS','ITC.NS','HINDALCO.NS','LTI.NS',	'INDUSINDBK.NS'	,'AMARAJABAT.NS',	'BERGEPAINT.NS'	,'GUJGASLTD.NS',	'HDFC.NS'	,'HDFCAMC.NS'	,
set = [
'HINDUNILVR.NS'	,
'IGL.NS'	,
'IRFC.NS'	,
'PETRONET.NS'	,
'BANDHANBNK.NS'	,
]
for i in range(len(set)):




    # Get the stock quote
    # name should be exact from yahoo finance site
    stock_name = set[i]
    df = pdr.DataReader(stock_name , data_source='yahoo',start='2000-01-01',end=datetime.datetime.now())

    # show the data 
    df


    #df.to_csv('price.csv') to save data as csv

    df.shape

    # Creating a new df for only close price 
    #data = df.filter(['Close','High'])           # gives 2 column 
    data = df.filter(['Close'])                   # gives 1 column
    data

    # create df to a  num py array         df.values : Only the values in the DataFrame will be returned, the axes labels will be removed.

    dataset = data.values
    dataset


    # Get the no of adta to train the model on say 80%
    training_data_len = math.ceil(len(dataset)*0.80)                    #Return the ceiling of x as an Integral.
    training_data_len

    # scale the data 
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(dataset)                        # range [0-1] on numpy array of dataset
    scaled_data




    # create a scaled training data set 
    train_data = scaled_data[0:training_data_len , : ]                # till row = training data len all column points

    # split the data into x_train  , and y_train data sets
    x_train = []                                                      # features = independent variables 
    y_train = []                                                      # lables = output = dependent variables 

    for i in range(60 , len(train_data)): 
        x_train.append(train_data[i-60:i , 0])                        # we are going to append 0 to 89th  values of train_data in 0th column
        y_train.append(train_data[i , 0])                             # lable/ prediction is 90th data point of 0th column from train_data to y_train

        if i <= 62:
            print(x_train)
            print()
            print(y_train)




    # convert x_train and y_train to numpy arrays so that array data can be provided to LSTM model
    x_train , y_train = np.array(x_train) , np.array(y_train)

    # x_train.shape               (2363, 30)   # after appending in every row tere are 30 columns of data 

    # x_train                                  # is 2d type row  and column data

    # reshaping the x_train data set as a LSTM model expects the input to be a three dimentionals array 
    x_train = np.reshape(x_train ,newshape= (x_train.shape[0] , x_train.shape[1] , 1))
    x_train.shape

    # Build the LSTM model 
    model = Sequential()

    # add a layer of 50 neuron , there output to be used in next layer thus return seq = true , since first layer thus shape of input neuron   , 
    # input shape = (time step , features ) = (90 days , 1 ) = (90 , close price)
    model.add(LSTM(50, activation ='relu', return_sequences=True, input_shape = (x_train.shape[1],1)))

    # add 1 last lstm layer but no lstm further thus return_sequence = False
    model.add(LSTM(50, return_sequences=False))

    # adding last dense layer          
    model.add(Dense(25))  

    # adding last dense layer          
    model.add(Dense(1)) 


    model.summary()



    # compile the model 
    # optimizer is used to minimise the loss fx 
    # loss fx is used to calculate what was the loss to see how well the model did on traiing 

    model.compile(optimizer='adam' , loss="mean_squared_error")




    # train the model
    model.fit(x=x_train,y=y_train,batch_size=1,epochs=10)



    model.save('{}_keras_model.h5'.format(stock_name))
    
    # creating the testing data set
    # create a new array containing scaled values from index  end of len(train_data) till end of the total data

    test_data = scaled_data[training_data_len-60 : , : ]         # all data from 60th to end of data with all column

    # creating a dataset x_test and y_test
    x_test = []
    y_test = dataset[training_data_len : , : ]                          # actual prediction values
                                                                        # dataset values are original closing prce and not scaled

    for i in range(60 , len(test_data)):
        x_test.append(test_data[i-60 : i, 0 ])



    # create the data to a numpy array as model need 3d array
    x_test = np.array(x_test)

    # reshape the data for the model      (newshape =no of rows = x_test.shape[0], timesteps = 60 days = x_test.shape[1] , and 1 )
    x_test = np.reshape(x_test , newshape= (x_test.shape[0] , x_test.shape[1] , 1 ))


    y_predictions = model.predict(x_test)    # scaled value




    # undo scaling
                                                               # we want the predictions to be exact same as real lables 
    y_predictions = scaler.inverse_transform(y_predictions)        # Undo the scaling of X according to feature_range(0-1)
                                                               # gaining back the real value

    # EVALUATE THE MODEL root mean square error (RMSE)
    # its a measure of how good the model predicts the response 
    # can be compared by std deviation less
    # lower value of rmse is better : try diff range of dataset 

    rmse = np.sqrt(np.mean(y_predictions - y_test)**2)
    print('Root mean square error in predicted vs actual values ', rmse)
    print("stock name " , stock_name )

[array([0.04604257, 0.04506337, 0.04206481, 0.04532757, 0.04993718,
       0.05061338, 0.05262905, 0.05861508, 0.06374571, 0.06062151,
       0.05678418, 0.05542438, 0.05422348, 0.05405535, 0.05493109,
       0.05605439, 0.05507704, 0.05507704, 0.05535787, 0.05403133,
       0.0509367 , 0.04675018, 0.04971548, 0.0508591 , 0.04749104,
       0.0460943 , 0.0460204 , 0.05012748, 0.04691276, 0.04859032,
       0.04577467, 0.04583749, 0.04617189, 0.04676311, 0.05363411,
       0.06106308, 0.06908694, 0.06704356, 0.07483464, 0.07684847,
       0.07368178, 0.06661125, 0.07128552, 0.06489672, 0.05661234,
       0.05216347, 0.05250711, 0.05286923, 0.04953627, 0.04808041,
       0.05386321, 0.05936703, 0.06725419, 0.06853453, 0.06853453,
       0.06853453, 0.06529579, 0.06106123, 0.05835459, 0.05734212])]

[0.053345898619059834]
[array([0.04604257, 0.04506337, 0.04206481, 0.04532757, 0.04993718,
       0.05061338, 0.05262905, 0.05861508, 0.06374571, 0.06062151,
       0.05678418, 0.05542438, 0.0